In [1]:
import os
import numpy as np
import pandas as pd
from consts import *
import tensorflow as tf
import autokeras as ak
from tensorflow.keras.models import load_model

In [3]:
df = pd.read_csv(modified_hotel_records_csv_file_path)
df.head()

,city,age_group,travel_with,total_female,total_male,purpose,tour_arrangement,package_transport,package_acc,package_guide,package_sight_see,night,total_cost
0,3,2,3,1,1,1,1,1,1,1,1,5,68526.0
1,5,0,4,1,3,5,0,0,0,0,0,12,113052.0
2,5,0,0,0,1,6,1,1,1,1,1,10,124263.0
3,1,0,3,1,1,1,1,1,1,0,0,5,56920.0
4,4,0,3,1,1,1,1,1,1,0,0,4,65998.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city               1055 non-null   int64  
 1   age_group          1055 non-null   int64  
 2   travel_with        1055 non-null   int64  
 3   total_female       1055 non-null   int64  
 4   total_male         1055 non-null   int64  
 5   purpose            1055 non-null   int64  
 6   tour_arrangement   1055 non-null   int64  
 7   package_transport  1055 non-null   int64  
 8   package_acc        1055 non-null   int64  
 9   package_guide      1055 non-null   int64  
 10  package_sight_see  1055 non-null   int64  
 11  night              1055 non-null   int64  
 12  total_cost         1055 non-null   float64
dtypes: float64(1), int64(12)
memory usage: 107.3 KB


In [5]:
# df = df.drop(['package_guide', 'package_acc', 'travel_with'], axis=1)
# df.head()

In [6]:
train_size = int(df.shape[0] * 0.95)
df[:train_size].to_csv(hotel_record_train_file_path, index=False)
df[train_size:].to_csv(hotel_record_test_file_path, index=False)

In [7]:
column_types={
    'city': 'categorical',
    'age_group': 'categorical',
    'travel_with': 'categorical',
    'total_female': 'numerical',
    'total_male': 'numerical',
    'purpose': 'categorical',
    'tour_arrangement': 'categorical',
    'package_transport': 'categorical',
    'package_acc': 'categorical',
    'package_guide': 'categorical',
    'package_sight_see': 'categorical',
    'night': 'numerical',
}

In [8]:
reg = ak.StructuredDataRegressor(
    overwrite=True,
    column_types=column_types,
    loss='mean_absolute_error',
    max_trials=5
)  
reg.fit(
    hotel_record_train_file_path,
    "total_cost",
    epochs=500,
)

Trial 5 Complete [00h 00m 31s]
val_loss: 90633.65625

Best val_loss So Far: 13337.8115234375
Total elapsed time: 00h 02m 00s
INFO:tensorflow:Oracle triggered exit
Epoch 1/500
32/32 [==============================] - 0s 716us/step - loss: 96496.1953 - mean_squared_error: 15117559808.0000
Epoch 2/500
32/32 [==============================] - 0s 588us/step - loss: 96479.9688 - mean_squared_error: 15114023936.0000
Epoch 3/500
32/32 [==============================] - 0s 737us/step - loss: 96432.3203 - mean_squared_error: 15103839232.0000
Epoch 4/500
32/32 [==============================] - 0s 532us/step - loss: 96322.5078 - mean_squared_error: 15080648704.0000
Epoch 5/500
32/32 [==============================] - 0s 513us/step - loss: 96109.1641 - mean_squared_error: 15035996160.0000
Epoch 6/500
32/32 [==============================] - 0s 503us/step - loss: 95745.3750 - mean_squared_error: 14960416768.0000
Epoch 7/500
32/32 [==============================] - 0s 532us/step - loss: 95182.8203 -

INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [9]:
predicted_y = reg.predict(hotel_record_test_file_path)
predicted_y

2/2 [==============================] - 0s 883us/step


array([[ 39238.703],
       [150874.48 ],
       [ 29342.33 ],
       [143028.53 ],
       [181683.8  ],
       [ 46632.832],
       [105102.03 ],
       [171618.95 ],
       [ 54870.125],
       [ 82594.9  ],
       [115162.57 ],
       [192317.81 ],
       [ 66416.945],
       [ 61370.785],
       [101976.78 ],
       [158256.92 ],
       [ 36655.49 ],
       [ 56688.355],
       [129718.22 ],
       [ 20893.066],
       [ 88249.91 ],
       [ 37857.297],
       [170610.92 ],
       [308266.   ],
       [203707.2  ],
       [134270.39 ],
       [ 40347.613],
       [ 36969.848],
       [156724.89 ],
       [216417.16 ],
       [206009.97 ],
       [ 82031.16 ],
       [ 80115.266],
       [ 76145.234],
       [ 95770.72 ],
       [109510.42 ],
       [ 32468.951],
       [141519.55 ],
       [130500.85 ],
       [102113.086],
       [ 59602.45 ],
       [ 65526.613],
       [106214.25 ],
       [192317.81 ],
       [209403.11 ],
       [ 79581.22 ],
       [ 41969.375],
       [ 7616

In [10]:
print(reg.evaluate(hotel_record_test_file_path, "total_cost"))

2/2 [==============================] - 0s 2ms/step - loss: 13455.3535 - mean_squared_error: 402359360.0000
[13455.353515625, 402359360.0]


In [36]:
model = reg.export_model()

try:
    model.save(regression_model_path_name, save_format="tf")
except Exception:
    model.save(regression_model_path_name + ".h5")

os.system("rm -rf ./structured_data_regressor")

INFO:tensorflow:Assets written to: model_autokeras_hotel/assets


INFO:tensorflow:Assets written to: model_autokeras_hotel/assets


In [41]:
loaded_model = load_model(regression_model_path_name, custom_objects=ak.CUSTOM_OBJECTS)
x_test = pd.read_csv(hotel_record_predict_file_path)

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

1/1 [==============================] - 0s 39ms/step
[[440976.9 ]
 [472006.53]]
